# Discretization
Used to bin numerical

In [10]:
import tensorflow as tf
age = tf.constant([[10.], [93.], [57.], [10.], [37.], [5.]])

discretize_layer = tf.keras.layers.Discretization(bin_boundaries=[10., 50.])
age_cats = discretize_layer(age)
display(age_cats)

tf.keras.backend.clear_session()

discretize_layer = tf.keras.layers.Discretization(num_bins=3)
discretize_layer.adapt(age)
age_cats = discretize_layer(age)
age_cats


<tf.Tensor: shape=(6, 1), dtype=int64, numpy=
array([[1],
       [2],
       [2],
       [1],
       [1],
       [0]])>

<tf.Tensor: shape=(6, 1), dtype=int64, numpy=
array([[1],
       [2],
       [2],
       [1],
       [2],
       [0]])>

# Category Encoding

In [11]:
one_hot = tf.keras.layers.CategoryEncoding(num_tokens=3)
one_hot(age_cats)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)>

# StringLookup

In [12]:
cities = ['Auckland', 'Paris', 'Paris', 'New York']
str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(cities)
str_lookup_layer([[c] for c in cities])

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[3],
       [1],
       [1],
       [2]])>

In [13]:
str_lookup_layer = tf.keras.layers.StringLookup(output_mode='one_hot')
str_lookup_layer.adapt(cities)
str_lookup_layer([[c] for c in cities])

<tf.Tensor: shape=(4, 4), dtype=int64, numpy=
array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])>

# Hashing

In [14]:
hashing_layer = tf.keras.layers.Hashing(num_bins=10)
hashing_layer([[c] for c in cities])

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[9],
       [0],
       [0],
       [8]])>